Batch processing

In [4]:
FILE_NAME='recipeNLG_with_entities.csv'

Set parameters

In [5]:
from math import floor

_ = !wc -l {FILE_NAME} # type: ignore
LINES = int(_[0].split()[0]) - 1

CHUNKSIZE=10_000
CHUNKS=floor(LINES/CHUNKSIZE)

Run in parallel processing

In [7]:
import requests
import pandas as pd
import io
import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

df_iter = pd.read_csv(FILE_NAME, iterator=True, chunksize=CHUNKSIZE)

def parallel_task(batch: pd.DataFrame):
    buff = io.BytesIO()
    batch.to_csv(buff, index=False)
    buff.seek(0)
    input_file = {'csv': (f'batch.csv', buff)}
    out = requests.post('http://localhost:5000/api/lexmapr', files=input_file)
 
    json = out.content.decode()
    with open('out.json', 'a') as file:
        file.write(json)

for batch in tqdm.tqdm(df_iter, total=CHUNKS):
    batch.parallel_apply(parallel_task)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


AttributeError: 'TextFileReader' object has no attribute 'parallel_apply'